### Ad Hoc Analytic
Best practice is to just know what you want to know from this 

In [1]:
import duckdb
from dotenv import load_dotenv
load_dotenv()

import os

con = duckdb.connect(os.getenv('DB_PATH'), read_only=True)
bs = con.sql('SELECT * FROM my.bank_statement')
bs.aggregate('''
             min(Date)::datetime::date as "Earliest Transaction", 
             max(Date)::datetime::date as "Latest Transaction",
             sum("Money out (£)") FILTER (WHERE Category != 'Rent & Essential')
                / datediff('month', min(Date), max(Date))
                as "Avg £ Out Per Month (Net Rent)",
             sum("Money in (£)")
                / datediff('month', min(Date), max(Date))
                as "Avg £ In",
             ''') # now the problem here is aggregate method in duckdb is different to ibis

┌──────────────────────┬────────────────────┬────────────────────────────────┬──────────┐
│ Earliest Transaction │ Latest Transaction │ Avg £ Out Per Month (Net Rent) │ Avg £ In │
│         date         │        date        │             double             │  double  │
├──────────────────────┼────────────────────┼────────────────────────────────┼──────────┤
│ 2023-06-01           │ 2023-11-30         │                         2561.8 │   3012.6 │
└──────────────────────┴────────────────────┴────────────────────────────────┴──────────┘

In [2]:
since_date = con.sql("select time_bucket(interval '2 months', today() - interval 1 month) as since_d")

print(since_date)
(
    con.sql("select * from my.bank_statement")
    .aggregate('''
               Category, 
               sum("Money Out (£)") Total,
               round(sum("Money Out (£)") 
                / datediff('Month',min(Date), max(Date)), 2)
                as "Per Month",
               sum("Money Out (£)")
                FILTER(WHERE Date::datetime::date >= (select since_d from since_date))
                as "Per Month Since"
               ''',
               "Category"
               )
)

┌────────────┐
│  since_d   │
│    date    │
├────────────┤
│ 2023-11-01 │
└────────────┘



┌──────────────────┬────────┬───────────┬─────────────────┐
│     Category     │ Total  │ Per Month │ Per Month Since │
│     varchar      │ double │  double   │     double      │
├──────────────────┼────────┼───────────┼─────────────────┤
│ Transport        │  467.0 │      93.4 │            41.0 │
│ Subscri & Apple  │ 1853.0 │     370.6 │            81.0 │
│ Rent & Essential │ 3406.0 │     681.2 │          1177.0 │
│ Food & Grocery   │ 1502.0 │     300.4 │           221.0 │
│ Cafe             │  582.0 │     116.4 │            38.0 │
│ Cloth & Shopping │  292.0 │      58.4 │            51.0 │
│ Other            │ 8113.0 │    1622.6 │          2302.0 │
└──────────────────┴────────┴───────────┴─────────────────┘

In [53]:
con